In [5]:
import pandas as pd

In [6]:
df=pd.read_csv('AB_test_data.csv')

In [7]:
# data frame
df

,purchase_TF,Variant,date,id
0,False,A,2019/11/8,0x25b44a
1,False,B,2020/8/27,0x46271e
2,False,A,2020/6/11,0x80b8f1
3,False,B,2020/8/22,0x8d736d
4,False,A,2020/8/5,0x96c9c8
...,...,...,...,...
129995,False,A,2020/7/23,0x4089c2
129996,False,A,2020/6/24,0x6a5e3a
129997,False,A,2019/10/12,0x95e302
129998,False,A,2020/3/18,0x7c4afa


In [8]:
# seperate variance A and vairance B
listType = df['Variant'].unique()
data_A =  df[df['Variant'].isin([listType[0]])]
data_B = df[df['Variant'].isin([listType[1]])]

In [9]:
data_B

,purchase_TF,Variant,date,id
1,False,B,2020/8/27,0x46271e
3,False,B,2020/8/22,0x8d736d
59,False,B,2020/8/19,0x3ff83f
74,False,B,2020/8/10,0x138d19
103,False,B,2020/8/4,0x966e6a
...,...,...,...,...
129805,False,B,2020/8/27,0x7d95d0
129827,False,B,2020/8/10,0x38a6e8
129879,False,B,2020/8/30,0x6a711a
129910,False,B,2020/8/13,0x13175a


In [10]:
# calculate P(1) for variance A
A_rate= data_A['purchase_TF'].value_counts().reset_index()
A_true=A_rate.iloc[1,1]
A_total=A_rate.iloc[1,1]+A_rate.iloc[0,1]

In [11]:
A_convert_Rate=A_true/A_total
print(A_convert_Rate)

0.149616


In [12]:
# calculate P(1) for variance B
B_rate= data_B['purchase_TF'].value_counts().reset_index()
B_true=B_rate.iloc[1,1]
B_total=B_rate.iloc[1,1]+B_rate.iloc[0,1]

In [13]:
B_convert_Rate=B_true/B_total
print(B_convert_Rate)

0.1766


## Q1 - A/B test - conversion rate

In [14]:
# calculate p-value
# see if the improvement is significant

import statsmodels.api as sm
z_scroe,p_value=sm.stats.proportions_ztest([A_true,B_true],[A_total,B_total],alternative='smaller')
z_scroe,p_value

(-5.2304877183522045, 8.453170449191526e-08)

## Q2 - Optimal sample size

In [15]:
import scipy as sc
from scipy.stats import norm

In [16]:
# calculate z-score of alpha (probability of type I error) 
z = sc.stats.norm.ppf(1-0.05/2)

In [17]:
z

1.959963984540054

In [18]:
# calculate z-scroe of beta (probability of type II error) 
b = sc.stats.norm.ppf(1-0.2)

In [19]:
b

0.8416212335729143

In [20]:
from math import sqrt

In [21]:
AB_convert_Rate = A_convert_Rate+B_convert_Rate

In [22]:
n=((z*(sqrt((AB_convert_Rate)*(1-(AB_convert_Rate/2))))+b*sqrt(A_convert_Rate*(1-A_convert_Rate)+B_convert_Rate*(1-B_convert_Rate))))**2/(A_convert_Rate-B_convert_Rate)**2

n=int(round(n))
n

2942

### Assume that A contains all the 125000 data points

In [44]:
A=[]
A_True=[]
A_sample = []
for i in range(0,10):
    sample=data_A.sample(n=n,replace=False)
    A_sample.append(sample)
    rate= sample['purchase_TF'].value_counts().reset_index()
    true=rate.iloc[1,1]
    total=rate.iloc[1,1]+rate.iloc[0,1]
    real_rate=true/total
    A.append(real_rate)
    A_True.append(true)

In [45]:
B=[]
B_True=[]
B_sample = []
for i in range(0,10):
    sample=data_B.sample(n=n,replace=False)
    B_sample.append(sample)
    rate= sample['purchase_TF'].value_counts().reset_index()
    true=rate.iloc[1,1]
    total=rate.iloc[1,1]+rate.iloc[0,1]
    real_rate=true/total
    B.append(real_rate)
    B_True.append(true)

In [46]:
for i in range(0,10):
    z_sample_scroe,p_sample_value=sm.stats.proportions_ztest([A_True[i],B_True[i]],[n,n],alternative='smaller')
    print(z_sample_scroe,p_sample_value)

-3.8537340041903367 5.816498773121778e-05
-4.098895846019702 2.0756288932503553e-05
-1.3950736840760354 0.08149681046270418
-2.027785103125133 0.021291092700855323
-3.302283535853394 0.00047950538164853154
-3.917841748930091 4.4672663691219165e-05
-3.8440251306838777 6.0516301681534884e-05
-3.2092319898369985 0.0006654502996016463
-1.7881653874672057 0.036874664373076745
-2.0480208198775265 0.020278979704986332


In [47]:
#from statsmodels.stats.power import zt_ind_solve_power
#from statsmodels.stats.proportion import proportion_effectsize as es
 
#zt_ind_solve_power(effect_size=es(prop1=0.149616, prop2=0.1766), alpha=0.05, power=0.8, alternative="two-sided")

## Q3

需按Q2修改


In [48]:
data_A_1_base = len(data_A[data_A['purchase_TF'] == True])
data_A_1_base

data_A_0_base = len(data_A[data_A['purchase_TF'] == False])
data_A_0_base

data_A_all_base = len(data_A['purchase_TF'])
data_A_all_base

125000

In [49]:
import math

# with most conservative boundary

a = 0.05
b = 0.2

# calculate boundary
ln_A = math.log(1/a)
ln_B = math.log(b)

In [50]:
for Whole_data in B_sample:
    
    data_A_all = data_A_all_base
    data_A_0 = data_A_0_base
    data_A_1 = data_A_1_base
    data_A_1_p = data_A_1 / data_A_all
    data_A_0_p = data_A_0 / data_A_all

    # let expected B convert rate be the same as Q1
    data_B_1_p = B_convert_Rate
    data_B_0_p = 1-B_convert_Rate

    ln_lamda_n = 0

    count_B = 0
    
    for index in range(len(Whole_data)):

        # if data belongs to group A
        if Whole_data.iloc[index,1] == 'A':

            pass


        # if data belongs to group B
        if Whole_data.iloc[index,1] == 'B':


            # if this point's purchase_TF is Ture
            if Whole_data.iloc[index,0] == True:

                ln_lamda_xi = math.log(data_B_1_p / data_A_1_p)
                ln_lamda_n = ln_lamda_n + ln_lamda_xi

                count_B = count_B + 1

            # if this point's purchase_TF is Ture
            if Whole_data.iloc[index,0] == False:

                ln_lamda_xi = math.log(data_B_0_p / data_A_0_p)

                ln_lamda_n = ln_lamda_n + ln_lamda_xi
                count_B = count_B + 1    

        # test if meet the boundary
        if ln_lamda_n >= ln_A:
            print('stop')
            print(count_B)
            print('accept H1')
            break

        elif ln_lamda_n <= ln_B:

            print('stop')
            print(count_B)
            print('reject H1')
            break

        else:
            pass

stop
1115
accept H1
stop
527
accept H1
stop
790
accept H1
stop
392
accept H1
stop
800
accept H1
stop
727
accept H1
stop
341
accept H1
stop
542
accept H1
stop
864
accept H1
stop
249
accept H1
